In [ ]:
import os
import sys

import tensorflow as tf 
import soundfile as sf
import yaml
import numpy as np
from  datetime import datetime
from IPython.display import Audio

from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoConfig
from tensorflow_tts.inference import AutoProcessor

print("Completed import Part")


In [7]:


# Initializing Vocoder-Model

melgan_config = AutoConfig.from_pretrained('model/melgan.v1.yaml')
melgan = TFAutoModel.from_pretrained(config=melgan_config , pretrained_path="model/melgan-1M6.h5")

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path='model/fastspeech2_quan.tflite')

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function for input data.
def prepare_input(input_ids):
  input_ids = tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0)
  return (input_ids,
          tf.convert_to_tensor([0], tf.int32),
          tf.convert_to_tensor([1.0], dtype=tf.float32),
          tf.convert_to_tensor([1.0], dtype=tf.float32),
          tf.convert_to_tensor([1.0], dtype=tf.float32))

#inferancing of the input data

def infer(input_text):
  processor = AutoProcessor.from_pretrained(pretrained_path="model/ljspeech_mapper.json")
  input_ids = processor.text_to_sequence(input_text.lower())
  interpreter.resize_tensor_input(input_details[0]['index'], 
                                  [1, len(input_ids)])
  interpreter.resize_tensor_input(input_details[1]['index'], 
                                  [1])
  interpreter.resize_tensor_input(input_details[2]['index'], 
                                  [1])
  interpreter.resize_tensor_input(input_details[3]['index'], 
                                  [1])
  interpreter.resize_tensor_input(input_details[4]['index'], 
                                  [1])
  interpreter.allocate_tensors()
  input_data = prepare_input(input_ids)
  for i, detail in enumerate(input_details):
    input_shape = detail['shape_signature']
    interpreter.set_tensor(detail['index'], input_data[i])

  interpreter.invoke()

  
  return (interpreter.get_tensor(output_details[0]['index']),
          interpreter.get_tensor(output_details[1]['index']))

print("End of inferacing")

end


In [8]:

#Input text

input_text = "Hello, Berlin bars will be allowed to welcome guests again starting on the upcoming Friday."

decoder_output_tflite, mel_output_tflite = infer(input_text)

audio_before_tflite = melgan(decoder_output_tflite)[0, :, 0]

audio_after_tflite = melgan(mel_output_tflite)[0, :, 0]

print("end of ids to melspectrogram")

end


In [10]:
#Generating Audio
Audio(data=audio_before_tflite, rate=22050)

In [11]:
#Generating Audio
Audio(data=audio_after_tflite, rate=22050)